In [1]:
# Load sql magic extension
%load_ext sql
# Connect to database
%sql mysql://root@127.0.0.1/current
# Import python modules
import numpy as np
import pandas as pd

### Define functions

In [2]:
def exec_statement(arg, df_out=0):
    ''' Accepts a string and executes it as a sql command '''
    %sql SET @query = :arg;
    %sql PREPARE stmt FROM @query;
    return_val = %sql EXECUTE stmt;
    %sql DEALLOCATE PREPARE stmt;
    if (df_out):
        return return_val.DataFrame()
    else:
        return return_val

def df_crossjoin(df1, df2, **kwargs):
    ''' Cross (or Cartesian)-joins two pandas dataframes '''
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1
    return_val = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    return_val.index = pd.MultiIndex.from_product((df1.index, df2.index))
    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)
    return return_val

### Define linking variables

In [3]:
# Define fields to compare tables by
tableA_fields = ['sex_id','n.id'] #pdd table  ,'zip_code'
tableB_fields = ['sex_id','newborn_id'] #bc table  ,'mothers_residence_zip_code'

# Define field(s) to block by
tableA_blocks = ['hospital_id','date_of_birth']
tableB_blocks = ['hospital_id','date_of_delivery']

# Define conditions on which to compare fields
conditions = [
    'cross_table[\'sex_id_x\']==cross_table[\'sex_id_y\']',
    'cross_table[\'id\']==cross_table[\'newborn_id\']']

### Blocking
Find all possible values for 1st blocking field ('hospital_id')

In [4]:
stmt = 'SELECT T.blks FROM (' + \
'SELECT DISTINCT ' + tableA_blocks[0] + ' AS blks ' \
'FROM ' +  tableA + ' UNION ' + \
'SELECT DISTINCT ' + tableB_blocks[0] + ' AS blks ' \
'FROM ' +  tableB + ') as T ORDER BY blks;'
block_list1 = exec_statement(stmt)

NameError: name 'tableA' is not defined

Find all possible values for 2nd blocking field ('date_of_delivery')

In [9]:
stmt = 'SELECT T.blks FROM (' + \
'SELECT DISTINCT ' + tableB_blocks[1] + ' AS blks ' \
'FROM ' +  tableB + ') as T ORDER BY blks;'
block_list2 = exec_statement(stmt)

0 rows affected.
0 rows affected.
1554 rows affected.
0 rows affected.


### Iterate over each block
- Create each block and export to python pandas dataframe
- Cross join the features-of-interest from both blocks (dataframes)

In [11]:
linked_match_count = [0]*len(tableA_fields)
match_count = [0]*len(tableA_fields)
pair_count = 0

i = 0
ii = 500


# Create 'blockA'
stmt1 = 'SELECT ' + ",".join(tableA_fields) + ' FROM patient_discharges p ' + \
'JOIN newborns n ON p.id=n.patient_discharge_id ' + \
'WHERE p.' + tableA_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
'AND p.admitted_on<=' + repr(str(block_list2[500][0])) + \
'AND p.discharged_on>=' + repr(str(block_list2[500][0])) + ';'
blockA = exec_statement(stmt1, df_out=1)

# Create 'blockB'
stmt2 = 'SELECT ' + ",".join(tableB_fields) + ' FROM births b ' + \
'JOIN deliveries d ON b.delivery_id=d.id' + \
'WHERE b.' + tableB_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
'AND b.' + tableB_blocks[1] + '=' + repr(str(block_list2[500][0])) + ';'
blockB = exec_statement(stmt2, df_out=1)


# Cross-join both blocks
cross_table = df_crossjoin(blockA, blockB)

# Count field matches and dump each into new column of dataframe
bool_table = pd.DataFrame(columns=tableA_fields)
for fieldA, cond in zip(tableA_fields, conditions):
    cond_res = eval(cond)
    bool_table[fieldA] = cond_res.values
    

match_count += bool_table.sum()
pair_count += bool_table.count()[0]
print(match_count)
print(pair_count)


0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
13 rows affected.


/usr/local/lib/python2.7/dist-packages/sqlalchemy/engine/default.py:470: Warning: Unknown table 'cross_table'
  cursor.execute(statement, parameters)


0 rows affected.
0 rows affected.
0 rows affected.
10 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
130 rows affected.
0 rows affected.
130 rows affected.
sex_id           68
newborn_pd_id     0
dtype: int64
130


In [18]:
linked_match_count = [0]*len(tableA_fields)
match_count = [0]*len(tableA_fields)
pair_count = 0

for i in range(len(block_list1)):
   for ii in range(len(block_list2)):

        # Drop tables, if they already exist
        drop_table('blockA')
        drop_table('blockB')
        drop_table('cross_table')

        # Create 'blockA' table
        stmt1 = 'CREATE TABLE blockA SELECT * FROM ' + tableA + ' a ' + \
        'WHERE a.' + tableA_blocks[0] + '=' + repr(str(block_list1[i][0])) + \
        ' AND a.admitted_on<' + repr(str(block_list2[ii][0])) + \
        ' AND a.discharged_on>' + repr(str(block_list2[ii][0])) + ';'
        exec_statement(stmt1)

        # Create 'blockB' table
        stmt2 = 'CREATE TABLE blockB SELECT * FROM ' + tableB + ' b ' + \
        'WHERE b.' + tableB_blocks[0] + '=' + repr(str(block_list1[i][0])) + \
        ' AND b.' + tableB_blocks[1] + '=' + repr(str(block_list2[ii][0])) + ';'
        exec_statement(stmt2)

        # Check that neither block is empty
        blockA_len = %sql SELECT COUNT(*) FROM blockA;
        blockB_len = %sql SELECT COUNT(*) FROM blockB;

        # If empty, skip to next iteration of loop
        if (blockA_len[0][0]==0) or (blockB_len[0][0]==0):
            continue
        
        # Prepare strings for sql statement based on the field (column) names within each table
        stmt1 = ''.join(['a.%s, ' % (tableA_fields[n]) for n in xrange(len(tableA_fields))])
        stmt2 = ''.join(['b.%s, ' % (tableB_fields[n]) for n in xrange(len(tableB_fields))])
        stmt2 = stmt2[:-2] # Drop the final comma

        # Create cross-joined table from both blocks
        cr_stmt = 'CREATE TABLE cross_table ' + 'SELECT a.id,' + stmt1 + stmt2 + \
        ', b.id AS birth_id FROM blockA a CROSS JOIN blockB b;'
        #  WHERE rand() <= .01
        print(cr_stmt)
        exec_statement(cr_stmt)
        
        # Dump sql table into python dataframe
        cross_table = %sql SELECT * FROM cross_table;
        cross_table_df = cross_table.DataFrame()

        # Count field matches and dump each into new column of dataframe
        bool_table = pd.DataFrame(columns=tableA_fields)
        for index,(fieldA, fieldB) in enumerate(zip(tableA_fields,tableB_fields)):
            bool_table[fieldA] = np.where(cross_table_df[fieldA]==cross_table_df[fieldB],1,0)

        match_count += bool_table.sum()
        pair_count += bool_table.count()[0]
        
print(match_count)
print(pair_count)


0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affecte

0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affecte

0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affecte

0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affecte

0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affecte

...

13 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
3 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXECUTE stmt;']
0 rows affected.
130 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
13 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
9 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXEC

17 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
3 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXECUTE stmt;']
0 rows affected.
130 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
9 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
6 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXECU

11 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
7 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXECUTE stmt;']
0 rows affected.
130 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
15 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
4 rows affected.
0 rows affected.
1 rows affected.
1 rows affected.
CREATE TABLE cross_table SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;
0 rows affected.
0 rows affected.
(_mysql_exceptions.OperationalError) (1050, "Table 'cross_table' already exists") [SQL: u'EXEC

14 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.


KeyboardInterrupt: 

In [15]:
# SQL WAY - Loop through fields provided, and count number of exact matches from both tables
match_sum = []
for fieldA, fieldB in zip(tableA_fields,tableB_fields):
    stmt = 'SELECT COUNT(*) FROM blockA a CROSS JOIN blockB b WHERE a.' + fieldA + '=b.' + fieldB + ';'
    result = exec_statement(stmt)
    match_sum.append(float(result[0][0]))
print(match_sum)

# Find denominator (total number of possible pairs)
tot_pair_cnt = %sql SELECT COUNT(p.id) FROM blockA a CROSS JOIN blockB b;
print(tot_pair_cnt)

0 rows affected.
0 rows affected.
1 rows affected.
0 rows affected.
[68.0]


In [13]:
stmt2

"CREATE TABLE blockB SELECT * FROM bc_table b WHERE b.hospital_id='1' AND b.date_of_delivery='2012-05-11';"

In [ ]:
CREATE TABLE pdd_table 
SELECT p.*, n.id AS newborn_pd_id, n.cesarean_section, n.birth_weight_group_id, n.plurality_group_id
FROM patient_discharges p
JOIN newborns n
ON p.id=n.patient_discharge_id;

CREATE TABLE bc_table 
SELECT * FROM births;

ALTER TABLE bc_table
CHANGE COLUMN id birth_id int(11) NOT NULL,
CHANGE COLUMN sex_id birth_sex int(11) NULL,
CHANGE COLUMN gestational_age_ob_estimate bc_gestational_age_ob_estimate int(11) NULL,
CHANGE COLUMN plurality bc_plurality int(11) NULL,
CHANGE COLUMN parity bc_parity int(11) NULL,
CHANGE COLUMN gestational_age_lmp bc_gestational_age_lmp int(11) NULL;

CREATE TABLE bc_join_table
SELECT *
FROM bc_table b
JOIN deliveries d
ON b.delivery_id=d.id;

SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id
FROM
(SELECT * 
FROM patient_discharges p
JOIN newborns n
ON p.id=n.patient_discharge_id
WHERE p.hospital_id='1'
AND p.admitted_on<'2012-05-11' AND p.discharged_on>'2012-05-11') a
CROSS JOIN
(SELECT * 
FROM births b
JOIN deliveries d
ON b.delivery_id=d.id
WHERE b.hospital_id='1' 
AND b.date_of_delivery='2012-05-11') b;

CREATE TABLE cross_table
SELECT a.id,a.sex_id, a.newborn_pd_id, b.birth_sex, b.newborn_id, b.id AS birth_id
FROM blockA a 
CROSS JOIN blockB b;
